In [1]:
import pandas as pd
import paddlehub as hub

In [2]:
data = pd.read_csv('../data/data_all.csv')

In [3]:
demo_set = data.loc[0:99, :]
demo_set

,time,comment,source,date
0,2022-06-19 15:42:32,董宇辉：小时候不懂得朱自清的《背影》，长大后才懂得,bilibili,2022-06-19
1,2022-06-19 15:42:32,活了18年了，别说初恋，我连男朋友都没有,bilibili,2022-06-19
2,2022-06-19 15:42:32,我的初恋就是我老公，谈了13年，结婚6个月，身边的人都觉得很不可思议，我们俩反而觉得挺正常的...,bilibili,2022-06-19
3,2022-06-19 16:42:32,她现在看到你如此成熟稳重有立场，她一定很高兴曾经成为你的女朋友，你是她的骄傲。,bilibili,2022-06-19
4,2022-06-19 16:42:32,成长,bilibili,2022-06-19
...,...,...,...,...
95,2022-06-19 08:33:14,请问顿顿每天什么时候直播啊,bilibili,2022-06-19
96,2022-06-19 14:33:14,好温柔,bilibili,2022-06-19
97,2022-06-19 12:33:14,他什么时候直播啊，在哪里直播，为什么b站上只有片段,bilibili,2022-06-19
98,2022-06-19 13:33:14,这才是教育,bilibili,2022-06-19


In [5]:
# paddlenlp：模型需要有标签数据微调，没用
# import paddlenlp
# import paddle
#
# MODEL_NAME_1 = 'ernie-1.0-base-zh'
# MODEL_NAME_2 = 'ernie-3.0-base-zh'
# ernie1_model = paddlenlp.transformers.ErnieModel.from_pretrained(MODEL_NAME_1)
# tokenizer = paddlenlp.transformers.ErnieTokenizer.from_pretrained(MODEL_NAME_1)
# encoded_text = tokenizer(text=demo_set['comment'])
# input_ids = paddle.to_tensor([encoded_text['input_ids']])

# 筛选某天数据
# data[data['date'] == '2022-06-10']
# data[data['date'] == '2022-06-16']

# paddlehub
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
module = hub.Module(name="ernie_skep_sentiment_analysis")
sentiment = module.predict_sentiment(demo_set['comment'], use_gpu=True)
sentiment

[2022-07-07 20:27:27,221] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object


MemoryError: (ResourceExhausted) Fail to alloc memory of 839684096 size.
  [Hint: p should not be null.] (at C:\home\workspace\Paddle_release\paddle\fluid\memory\detail\system_allocator.cc:70)
  [operator < fc > error]

In [ ]:
sentiment_df = pd.DataFrame(sentiment)
sentiment_df

In [ ]:
demo_set_sentiment = pd.concat([demo_set, sentiment_df], axis=1)
demo_set_sentiment

In [ ]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment')
model = BertForSequenceClassification.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment')

In [ ]:
import numpy as np
comment = np.array(demo_set['comment']).tolist()
text = '到底要看多少书才能有这样的输出'
output = model(torch.tensor(tokenizer(comment, padding=True)['input_ids']).to('cuda'))
print(torch.nn.functional.softmax(output.logits, dim=-1))

In [ ]:
output.